In [111]:
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.disable_max_rows()
from joblib import Parallel, delayed
from numpy.random import Generator, PCG64
rng = Generator(PCG64())
import random
import math
import scipy.stats as stats

In [2]:
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

# Load Cleaned Version of Data

In [81]:
responses = pd.read_csv("https://data-visualization-benchmark.s3.us-west-2.amazonaws.com/vt-fusion/participant_responses.csv")
performance_df = responses.copy()
print("Number of participants in cleaned dataset: ", len(responses[['participant_id']].value_counts()))

Number of participants in cleaned dataset:  426


# Helpers

### Substitute task categories

Below is the procedure used to define a simpler common set of task types that could apply to all assessments:
- value identification, where participants need to retrieve an individual value appearing in a plot (e.g., finding
the maximum value);
- arithmetic computation, where participants are expected to perform arithmetic operations over multiple values displayed in the plot (e.g., finding
the average of two values); 
- statistical inference, where participants are required to estimate latent parameters based on the values shown (e.g., judge the strength of trends or presence of clusters).

In [82]:
vlat_format_map = {
    'find_extremum': 'value identification', 
    'determine_Range': 'value identification', 
    'retrieve_value': 'value identification',
    'make_comparisons': 'arithmetic computation',
    'find_correlations_trends': 'statistical inference', 
    'characterize_distribution': 'statistical inference',
    'find_anomolies': 'statistical inference', 
    'find_clusters': 'statistical inference', 
}

calvi_format_map = {
    'Make Comparisons': 'arithmetic computation',
    'Find Correlations/Trends': 'statistical inference', 
    'Find Extremum': 'value identification',
    'Retrieve Value': 'value identification', 
    'Make Predictions': 'statistical inference', 
    'Aggregate Values': 'statistical inference'
}

brbf_task_map = {
    'max': "value identification", 
    'min': 'value identification', 
    'trend': 'statistical inference',
    'trendComp': 'arithmetic computation', 
    'average': 'statistical inference', 
    'intersection': 'value identification',
}

wainer_question_map = {
    "How many months have more rain than the average month?": "statistical inference",
    "Which season has more rain, the summer or the spring?": "arithmetic computation",
    "Which season has the most rain?": "value identification",
    "In San Francisco, as the weather gets warmer, there is generally more rain.": "statistical inference",
    "In which season does each month have less rain than the month before?": "arithmetic computation",
    "How many months have less than 40mm of rain?": "arithmetic computation",
    "How much does it rain in March?": "value identification",
    "Which month has 25 mm of rain?": "value identification"
}

ggr_question_map = {
    # from level 1
    "Approximately what percentage of people had Adeolitis in the year 2000?": "value identification",
    "What percentage of patients recovered after chemotherapy?": "value identification",
    "Of 100 patients with disease X, how many are women?": "value identification",
    "Of all the people who die from cancer, approximately what percentage dies from lung cancer?": "value identification",

    # from level 2
    "Approximately what percentage of people who die from cancer die from colon cancer, breast cancer, and prostate cancer taken together?": "arithmetic computation",
    "How many more men than women are there among 100 patients with disease X?": "arithmetic computation",
    "What is the difference between the percentage of patients who recovered after a surgery and the percentage of patients who recovered after radiation therapy?": "arithmetic computation",
    "When was the increase in the percentage of people with Adeolitis higher?": "arithmetic computation",

    # from level 3
    "What percentage of patients recovered after chemotherapy?": "value identification",
    "According to your best guess, what will the percentage of people with Adeolitis be in the year 2010?": "statistical inference",
    "Between 1980 and 1990, which disease had a higher increase in the percentage of people affected?": "statistical inference",
    "Compared to the placebo, which treatment leads to a larger decrease in the percentage of patients who die?": "statistical inference",
    "What is the percentage of cancer patients who die after chemotherapy?": "value identification",
    "Which of the treatments contributes to a larger decrease in the percentage of sick patients?": "statistical inference"
}

task_category_map = ggr_question_map | wainer_question_map | brbf_task_map | vlat_format_map | calvi_format_map

### Substitute chart categories

Below is a mapping to extract graph labels from a the ones defined in each test

In [7]:
chart_categories = {
    'Dot Plot': 'Dot Plot',
    'Line Chart': 'Line',
    'Pie Chart': 'Pie',
    'chart': 'Table',
    'radial': 'Radial',
    'line': 'Line',
    'bar': 'Bar',
    'Scatterplot': 'Scatter',
    'Bar chart': 'Bar',
    'Stacked area chart': 'Stacked Area',
    '100% stacked bar chart': '100% Stacked Bar',
    'Line chart': 'Line',
    'Choropleth map': 'Map',
    'Area chart': 'Area',
    'Choropleth Map': 'Map',
    'Pie chart': 'Pie',
    'Stacked bar chart': 'Stacked Bar',
    'bg-table': 'Table',
    'bg': 'Bar',
    'sp-table': 'Table',
    'sp': 'Scatter',
    'lg2': 'Line',
    'lg1-table': 'Table',
    'lg1': 'Line',
    'lg2-table': 'Table',
    'Bar Chart': 'Bar',
    'Bubble Chart': 'Scatter',
    'Area Chart': 'Area',
    '100 % Stacked Bar Chart': '100% Stacked Bar',
    'Stacked Area Chart': 'Stacked Area',
    'Stacked Bar Chart': 'Stacked Bar',
    'Treemap': 'Treemap',
    'table': 'Table'
}

### Create item dataframe

In [8]:
def create_item_df():
    tests = ['wainer', 'brbf', 'ggr-mc', 'vlat', 'calvi']
    item_df = []
    for test in tests:
        idf = pd.read_csv(f'https://data-visualization-benchmark.s3.us-west-2.amazonaws.com/{test}/questions.csv')
        idf['test_type'] = test
        item_df.append(idf)

    item_df = pd.concat(item_df)
    item_df['question_image'] = item_df['question'] + " + " + item_df['image_file']
    return item_df
    
def add_brbf_categories(r):
    if (r['test_type'] == 'brbf'):
        return '-'.join(r['image_file'].split("-")[:-1])
    return r['chart_type']


item_df = create_item_df()
item_df['chart_type_filled'] = item_df.apply(add_brbf_categories, axis=1)
item_df['chart_type'] = item_df['chart_type_filled'].replace(chart_categories)
# item_df = item_df[item_df['chart_type'] != 'Table']

item_df['task_category'] = item_df['task_category'].apply(
    lambda t : task_category_map[t] if t in task_category_map.keys() else t
)
item_df['task_category'] = item_df.apply(
    lambda r : task_category_map[r['question']] if r['question'] in task_category_map.keys() else r['task_category'],
    axis=1
)
item_df['task_category'] = item_df['task_category'].apply(lambda x : x.replace(" ", "-")).to_numpy()
item_df['chart_type'] = item_df['chart_type'].apply(lambda x : x.replace(" ", "-")).to_numpy()

item_df['item_id'] = item_df['question_image']

In [9]:
def bootstrap_ci(
        data: pd.DataFrame,
        measure,
        id_col,
        n_iterations=1000,
        statistic=np.mean
    ):
    """
    
    """
    
    items = list(data[id_col].unique())
    n_size = len(items)
    df = data.copy()

    def bootstrap_iteration(data, chosen_items):
        # filter_df = data[data[id_col].isin(chosen_items)] # Filter based on chosen questions
        filter_df = pd.concat(
            [data[data[id_col].isin([item])] for item in chosen_items]
        )
        
        bs_mean = statistic(filter_df[measure]) 
        return (bs_mean, list(chosen_items))

    qset_means = Parallel(n_jobs=-1)(
        delayed(bootstrap_iteration)(
            df.copy(),
            rng.choice(items, n_size,  replace=True)
        ) for _ in range(n_iterations)
    )
    
    means = []
    qs_used = []
    means = [bs_mean for bs_mean, chosen_qs in qset_means]
 
    # 95% confidence interval
    lower = np.percentile(means, 2.5)
    upper = np.percentile(means, 97.5)
    
    return lower, upper

def create_confidence_interval_df(
    data: pd.DataFrame,
    measure, 
    id_col,
    condition_col,
    statistic=np.mean
):
    """ create the dataframe for 95% bootstrapped confidence interval
    data: dataset
    measure: dependent variable
    id_col: units to bootstrap along (e.g. participant_d or item_id)
    condition_col: the different conditions of the experiments
    """
    
    data_list = []

    for condition in data[condition_col].unique():
        condition_data = data[data[condition_col] == condition]

        lower, upper = bootstrap_ci(condition_data, measure=measure, statistic=statistic, id_col=id_col)
        mean = condition_data[measure].mean()
        
        data_list.append({
            "category": condition,
            "mean": mean,
            "ci_upper": upper, 
            "ci_lower": lower,
        })

    ci_df = pd.DataFrame(data_list)

    item_level_data = data.rename(columns={
        condition_col: 'category', 
    }).groupby([id_col, 'category'])[measure].mean().reset_index()

    return item_level_data, ci_df

In [10]:
assessment_color_map = {
    'wainer': '#3a0ca3',
    'ggr-mc': '#e26d5c',
    'brbf': '#f4a261',
    'vlat': '#247ba0',
    'calvi': '#70c1b3'
}


def box_muller():
    '''
    apparently controls how much jitter but who knows :shrug:
    '''
    # jitter_level * math.sqrt(-2 * math.log(random.random())) #* math.cos(2 * math.pi * random.random())
    return rng.normal() 


In [99]:
def find_amount_at_chance(r):
    option_count = 0
    for i in range(1, 11):
        if not pd.isna(r[f'mc{i}']):
            option_count += 1
    return 1 / option_count

item_df['chance_selection'] = item_df.apply(find_amount_at_chance, axis=1)

# Performance across categories

## How does performance vary across items?

Code for Figure 3: Average performance across all items. Items belonging to the same test share the same color. Error bars represent
bootstrapped 95% confidence intervals

In [94]:
try:
    print("Reloading item-performance dataframes")
    item_ci_df = pd.read_csv("./dataframes/item_ci_df.csv")
    item_participant_level_data = pd.read_csv("./dataframes/item_participant_level_data.csv")
    
except:
    print("Regenerating item-performance dataframes")
    item_participant_level_data, item_ci_df = create_confidence_interval_df(
        performance_df, 
        measure='is_correct',
        id_col='participant_id',
        condition_col='question_image',
    )
    item_ci_df = pd.merge(item_ci_df, item_df[['question_image', 'test_type']].rename(columns={'question_image': 'category'}))
    item_participant_level_data.to_csv("./dataframes/item_participant_level_data.csv")
    item_ci_df.to_csv("./dataframes/item_ci_df.csv")

Reloading item-performance dataframes


In [102]:
assessment_color_map = {
    'wainer': '#8947BF', # '#3a0ca3',
    'ggr-mc': '#e26d5c',
    'brbf': '#f4a261',
    'vlat': '#247ba0',
    'calvi': '#70c1b3'
}

item_ci_df_sorted = item_ci_df.sort_values(by='mean', ascending=False)
item_domain_order = item_ci_df_sorted['category'].to_list()

test_domain = list(assessment_color_map.keys())
test_domain_color = list(assessment_color_map.values())

def create_item_accuracy_chart(ci_df, domain_order):
    """ Generate chart for point estimate and confidence intervals overlayed with item level scatteplots
    """
    # Create dot+error bar plot
    chart = alt.Chart(ci_df).mark_bar(opacity=0.7).encode(
        x=alt.X('category:N', title='Category', scale=alt.Scale(domain=domain_order), axis=None),
        y=alt.Y('mean:Q', title='Prop. Correct'),
        color=alt.Color('test_type:N', scale=alt.Scale(domain=test_domain, range=test_domain_color), legend=None),
    ) + alt.Chart(ci_df).mark_rule(strokeWidth=0.4, opacity=1).encode(
        x=alt.X('category:N'),
        y=alt.Y('ci_lower:Q'),
        y2='ci_upper:Q',
        color=alt.value("#717d7e")
    )
    
    return chart.properties(width=800, height=200)

Figure 3

In [103]:
item_performance_reliability_plot = create_item_accuracy_chart(
    item_ci_df, 
    domain_order=item_domain_order,
)
item_performance_reliability_plot.save("./figures/item_performance_reliability.pdf")
item_performance_reliability_plot

alt.LayerChart(...)

In [98]:
item_ci_df.sort_values(by='mean')

,Unnamed: 0,category,mean,ci_upper,ci_lower,test_type
195,195,Does cell phone brand A have more than half of...,0.012048,0.036145,0.000000,calvi
204,204,Does any members of species C weigh more than ...,0.023256,0.058140,0.000000,calvi
197,197,Was the average amount of precipitation over 1...,0.034091,0.068466,0.000000,calvi
87,87,Which planet has the lowest life expectancy? +...,0.048193,0.096386,0.012048,brbf
201,201,"Assuming today is May 01, 2022, which of the f...",0.048780,0.097561,0.012195,calvi
...,...,...,...,...,...,...
123,123,In which country is the average internet speed...,0.976190,1.000000,0.940476,vlat
34,34,Which planet has the highest life expectancy? ...,0.976190,1.000000,0.940476,brbf
82,82,Do all of these planets have the same life exp...,0.988095,1.000000,0.964286,brbf
135,135,In which company is the global smartphone mark...,0.988235,1.000000,0.964706,vlat


Trying prop. correct per item but removed by 

In [71]:
item_ci_df = pd.merge(item_df[['item_id', 'chance_selection']], item_ci_df, left_on='item_id', right_on='category')
item_ci_df['mean'] = item_ci_df['mean'] - item_ci_df['chance_selection']
item_ci_df['ci_upper'] = item_ci_df['ci_upper'] - item_ci_df['chance_selection']
item_ci_df['ci_lower'] = item_ci_df['ci_lower'] - item_ci_df['chance_selection']

item_performance_reliability_below_chance_plot = create_item_accuracy_chart(
    item_ci_df, 
    domain_order=item_domain_order,
)
item_performance_reliability_below_chance_plot

alt.LayerChart(...)

In [72]:
item_ci_df.sort_values(by='mean').iloc[0]['item_id']

'Does cell phone brand A have more than half of the total market share? + T28.png'

## How does performance vary across tests?

Code for Figure 4A: Performance across different tests measured by the mean proportion of correct responses. Opaque dots indicate the mean proportion of correct responses for individual items. Error bars represent bootstrapped 95% confidence intervals.

In [104]:
try:
    print("Reloading test-performance dataframe")
    test_ci_df = pd.read_csv("./dataframes/test_ci_df.csv")
    test_item_level_data = pd.read_csv("./dataframes/test_item_level_data.csv")
    # raise Error
except:
    print("Regenerating test-performance dataframe")
    test_item_level_data, test_ci_df = create_confidence_interval_df(
        performance_df, 
        measure='is_correct',
        id_col='question_image',
        condition_col='test_type',
    )
    test_item_level_data.to_csv("./dataframes/test_item_level_data.csv", index=False)
    test_ci_df.to_csv("./dataframes/test_ci_df.csv", index=False)

Reloading test-performance dataframe


In [105]:
def transform_ci_df(original_ci_df, item_df):
    ci_df = pd.merge(item_df[['item_id', 'chance_selection']], original_ci_df, left_on='item_id', right_on='category').copy()
    ci_df['mean'] = ci_df['mean'] - ci_df['chance_selection']
    ci_df['ci_upper'] = ci_df['ci_upper'] - ci_df['chance_selection']
    ci_df['ci_lower'] = ci_df['ci_lower'] - ci_df['chance_selection']
    return ci_df

transform_ci_df(test_ci_df, item_df)

,item_id,chance_selection,category,mean,ci_upper,ci_lower


In [106]:
test_ci_df

,category,mean,ci_upper,ci_lower
0,wainer,0.777124,0.840152,0.711434
1,brbf,0.687439,0.733594,0.633982
2,ggr-mc,0.616352,0.752746,0.478788
3,vlat,0.643270,0.711412,0.576438
4,calvi,0.414917,0.483861,0.347289


In [107]:
test_order = list(assessment_color_map.keys())
test_domain_colors = list(assessment_color_map.values())

def create_test_accuracy_chart(ci_df, item_level_data, domain_order, test_domain_colors=[]):
    """ Generate chart for point estimate and confidence intervals overlayed with item level scatteplots
    """
    # Create dot+error bar plot
    chart = alt.Chart(ci_df).mark_point(filled=True, size=75, opacity=1).encode(
        x=alt.X('category:N', title='Category', scale=alt.Scale(domain=domain_order)),
        y=alt.Y('mean:Q', title='Prop. Correct'),
        color=alt.Color('category:N', scale=alt.Scale(domain=domain_order, range=test_domain_colors), legend=None),
    ) + alt.Chart(ci_df).mark_rule(strokeWidth=2).encode(
        x=alt.X('category:N'),
        y=alt.Y('ci_lower:Q'),
        y2='ci_upper:Q',
        color=alt.Color('category:N', legend=None),
        opacity=alt.value(1)
    )
    
    scatter_plot = alt.Chart(item_level_data).mark_point(filled=True).encode(
        x=alt.X('category:N', title='Category'),
        y=alt.Y("is_correct:Q",),
        xOffset="jitter:Q",
        color=alt.Color('category:N', legend=None),
        size=alt.value(16),
        opacity=alt.value(0.3)
    )
    return (chart + scatter_plot)

test_item_level_data['jitter'] = test_item_level_data.apply(
    lambda _ : box_muller(),
    axis=1
)

test_type_performance = create_test_accuracy_chart(
    test_ci_df, test_item_level_data, test_order, test_domain_colors
).properties(
    width=40*5, height=200, title=f"Prop. correct across tests", 
)
test_type_performance.save("./figures/test_type_performance.pdf")
test_type_performance

alt.LayerChart(...)

In [108]:
test_ci_df.sort_values(by="mean")

,category,mean,ci_upper,ci_lower
4,calvi,0.414917,0.483861,0.347289
2,ggr-mc,0.616352,0.752746,0.478788
3,vlat,0.643270,0.711412,0.576438
1,brbf,0.687439,0.733594,0.633982
0,wainer,0.777124,0.840152,0.711434


In [110]:
total_chances = []
for i, row in item_df[item_df['test_type'] == 'calvi'].iterrows():
    valid_mc_counter = 0
    for i in range(1, 11):
        
        if not pd.isna(row[f'mc{i}']):
            valid_mc_counter += 1

    total_chances.append(1 / valid_mc_counter)
np.mean(total_chances)

0.2833333333333333

In [112]:
item_means = performance_df.groupby(["test_type", 'question_image'])['is_correct'].mean().reset_index()

groups = [performance_df["is_correct"].values for _, group in item_means.groupby('test_type')]
f_statistic, p_value = stats.f_oneway(*groups)

In [114]:
# item_means

## How does performance vary across different types of graphs?

Code for Figure 4C: Performance across different graph types measured by the mean proportion of correct responses. Opaque dots indicate the mean proportion of correct responses for individual items. Error bars represent bootstrapped 95% confidence intervals.

In [115]:
try:
    print("Reloading test-performance dataframe")
    graph_item_level_data = pd.read_csv("./dataframes/graph_item_level_data.csv")
    graph_ci_df = pd.read_csv("./dataframes/graph_ci_df.csv")
except:
    print("Regenerating test-performance dataframe")
    
    def add_brbf_categories(r):
        if (r['test_type'] == 'brbf'):
            return '-'.join(r['image_file'].split("-")[:-1])
        return r['chart_type']

    chart_performance_df = performance_df.copy()
    chart_performance_df['chart_type_filled'] = chart_performance_df.apply(add_brbf_categories, axis=1)
    chart_performance_df['common_chart_type'] = chart_performance_df['chart_type_filled'].replace(chart_categories)
    chart_performance_df = chart_performance_df[chart_performance_df['common_chart_type'] != 'Table']
    
    graph_item_level_data, graph_ci_df = create_confidence_interval_df(
        chart_performance_df, 
        measure='is_correct',
        id_col='question_image',
        condition_col='common_chart_type',
    )
    graph_item_level_data = pd.merge(graph_item_level_data, item_df[['test_type', 'question_image']])
    graph_ci_df.to_csv("./dataframes/graph_ci_df.csv",index=False)
    graph_item_level_data.to_csv("./dataframes/graph_item_level_data.csv",index=False)

Reloading test-performance dataframe


In [116]:
test_order = list(assessment_color_map.keys())
test_domain_colors = list(assessment_color_map.values())
chart_order = graph_ci_df.sort_values(by="mean", ascending=False)['category'].to_list()

def format_hexcode_alpha(hexcode):
    samples = np.linspace(60, 99, num=len(chart_order))
    return [hexcode + str(int(s)) for s in samples][::-1]

# color_order = [hexcode for hexcode in format_hexcode_alpha("#17202a")]
color_order = ["#17202a" for _ in range(len(chart_order))]

def create_graph_performance_chart(ci_df, item_level_data, domain_order, domain_color=[]):
    """ Generate chart for point estimate and confidence intervals overlayed with item level scatteplots
    """
    # Create dot+error bar plot
    chart = alt.Chart(ci_df).mark_point(filled=True, size=75, opacity=1).encode(
        x=alt.X('category:N', title='Category', scale=alt.Scale(domain=domain_order)),
        y=alt.Y('mean:Q', title='Prop. Correct'),
        color=alt.Color('category:N', scale=alt.Scale(domain=domain_order, range=domain_color), legend=None),
    ) + alt.Chart(ci_df).mark_rule(strokeWidth=2).encode(
        x=alt.X('category:N'),
        y=alt.Y('ci_lower:Q'),
        y2='ci_upper:Q',
        color=alt.Color('category:N', scale=alt.Scale(domain=domain_order, range=domain_color), legend=None)
    )
    
    scatter_plot = alt.Chart(item_level_data).mark_point(opacity=0.35, filled=True).encode(
        x=alt.X('category:N', title='Category'),
        y=alt.Y("is_correct:Q",),
        xOffset="jitter:Q",
        color=alt.Color('test_type:N', scale=alt.Scale(domain=test_order, range=test_domain_colors), legend=None),
        size=alt.value(16),
    )

    return (chart + scatter_plot).resolve_scale(color='independent')


graph_item_level_data['jitter'] = graph_item_level_data.apply(
    lambda _ : box_muller(),
    axis=1
)
chart_performance = create_graph_performance_chart(graph_ci_df, graph_item_level_data, chart_order, color_order).properties(
    width=13*34, height=200, title=f"Prop. correct across tasks", 
)
chart_performance.save("./figures/graph_type_performance.pdf")
chart_performance

alt.LayerChart(...)

In [118]:
graph_ci_df.sort_values(by='mean')

,category,mean,ci_upper,ci_lower
6,Stacked Bar,0.395299,0.509522,0.279650
10,Stacked Area,0.397059,0.672735,0.207292
11,Map,0.465766,0.658661,0.278163
9,Area,0.510355,0.704065,0.316322
3,Scatter,0.532263,0.607401,0.449259
7,100% Stacked Bar,0.574419,0.711628,0.450116
0,Line,0.598045,0.677190,0.516103
4,Pie,0.637011,0.834148,0.414683
1,Bar,0.694978,0.773654,0.610742
5,Dot Plot,0.747126,0.896552,0.597701


## How does performance vary across different kinds of tasks?

Code for Figure 4B: Performance across different task types measured by the mean proportion of correct responses. Opaque dots indicate the mean proportion of correct responses for individual items. Error bars represent bootstrapped 95% confidence intervals.

In [119]:
try:
    print("Reloading task-performance dataframe")
    task_item_level_data = pd.read_csv("./dataframes/task_item_level_data.csv")
    task_ci_df = pd.read_csv("./dataframes/task_ci_df.csv")
except:
    print("Regenerating task-performance dataframe")
    task_performance_df = performance_df.copy()
    task_performance_df['task_category'] = task_performance_df['task_category'].apply(
        lambda t : task_category_map[t] if t in task_category_map.keys() else t
    )
    task_performance_df['task_category'] = task_performance_df.apply(
        lambda r : task_category_map[r['question']] if r['question'] in task_category_map.keys() else r['task_category'],
        axis=1
    )
    # question_task_df = task_performance_df[['question_image', 'task_category']].value_counts().reset_index()
    # question_task_df[['task_category']].value_counts().reset_index()
    
    task_item_level_data, task_ci_df = create_confidence_interval_df(
        task_performance_df, 
        measure='is_correct',
        id_col='question_image',
        condition_col='task_category',
    )
    task_item_level_data = pd.merge(task_item_level_data, item_df[['test_type', 'question_image']])
    task_item_level_data = task_item_level_data.reset_index()

    task_item_level_data.to_csv("./dataframes/task_item_level_data.csv", index=False)
    task_ci_df.to_csv("./dataframes/task_ci_df.csv", index=False)

Reloading task-performance dataframe


In [120]:
task_performance_df = performance_df.copy()
task_performance_df['task_category'] = task_performance_df['task_category'].apply(
    lambda t : task_category_map[t] if t in task_category_map.keys() else t
)
task_performance_df['task_category'] = task_performance_df.apply(
    lambda r : task_category_map[r['question']] if r['question'] in task_category_map.keys() else r['task_category'],
    axis=1
)
task_performance_df[['test_type', 'task_category']].value_counts()

test_type  task_category         
brbf       value identification      3062
calvi      arithmetic computation    2717
vlat       value identification      2529
calvi      statistical inference     2207
brbf       statistical inference     2026
vlat       arithmetic computation    1033
wainer     arithmetic computation    1024
           value identification      1011
brbf       arithmetic computation    1010
vlat       statistical inference      940
wainer     statistical inference      684
ggr-mc     value identification       436
           arithmetic computation     341
           statistical inference      336
calvi      value identification       171
Name: count, dtype: int64

In [121]:
task_order = task_ci_df.sort_values(by="mean", ascending=False)['category'].to_list()
test_order = list(assessment_color_map.keys())
test_domain_colors = list(assessment_color_map.values())
task_color_order = ["#17202a" for _ in range(len(task_order))]

def create_task_performance_chart(ci_df, item_level_data, domain_order, domain_color=[]):
    """ Generate chart for point estimate and confidence intervals overlayed with item level scatteplots
    """
    # Create dot+error bar plot
    chart = alt.Chart(ci_df).mark_point(filled=True, size=75).encode(
        x=alt.X('category:N', title='Category', scale=alt.Scale(domain=domain_order)),
        y=alt.Y('mean:Q', title='Prop. Correct'),
        color=alt.Color('category:N', scale=alt.Scale(domain=domain_order, range=domain_color), legend=None),
        opacity=alt.value(1)
    ) + alt.Chart(ci_df).mark_rule(strokeWidth=2).encode(
        x=alt.X('category:N'),
        y=alt.Y('ci_lower:Q'),
        y2='ci_upper:Q',
        color=alt.Color('category:N', scale=alt.Scale(domain=domain_order, range=domain_color), legend=None),
        opacity=alt.value(1)
    )

    scatter_plot = alt.Chart(item_level_data).mark_point(filled=True).encode(
        x=alt.X('category:N', title='Category'),
        y=alt.Y("is_correct:Q",),
        xOffset="jitter:Q",
        color=alt.Color('test_type:N', scale=alt.Scale(domain=test_order, range=test_domain_colors), legend=None),
        size=alt.value(16),
        opacity=alt.value(0.3)
    )    
    return (scatter_plot + chart).resolve_scale(color='independent')

task_item_level_data['jitter'] = task_item_level_data.apply(
    lambda i : box_muller(),
    axis=1
)
task_performance = create_task_performance_chart(
    task_ci_df, task_item_level_data, task_order, task_color_order).properties(
    width=40*3, height=200, title=f"Prop. correct across tasks", 
)
task_performance.save("./figures/task_type_performance.pdf")
task_performance

alt.LayerChart(...)

In [122]:
task_ci_df

,category,mean,ci_upper,ci_lower
0,value identification,0.710917,0.766009,0.650338
1,arithmetic computation,0.516408,0.577568,0.452820
2,statistical inference,0.599548,0.654147,0.539598


## How does performance vary across presentation modality (tables vs. plots)?

Code for Figure 6: Performance across different data presentation formats (i.e. table or data visualization). Each line segment
represents a pair of items in either WAN and BRBF which differ only in data presentation format. Error bars represent bootstrapped 95% confidence intervals.

### Preprocess

In [125]:
modality_performance_df = performance_df.copy()
modality_performance_df = modality_performance_df[
    modality_performance_df['test_type'].isin(['brbf', 'wainer'])
]

modality_performance_df['chart_type'] = modality_performance_df.apply(
    lambda r : r['graph_type'] if pd.isna(r['chart_type']) else r['chart_type'],
    axis=1
)
modality_performance_df['chart_type'] = modality_performance_df['chart_type'].astype(str)
modality_performance_df['modality'] = modality_performance_df.apply(
    lambda r : "table" if "table" in r['chart_type'] else "graph",
    axis=1
)
modality_question_count = modality_performance_df[['question_image', 'modality']].value_counts().reset_index()
modality_count = modality_question_count[['modality']].value_counts().reset_index()
modality_count

,modality,count
0,graph,60
1,table,44


In [126]:
wainer_gt_performance_df = modality_performance_df[
    modality_performance_df['test_type'].isin(['wainer'])
].copy()

wainer_gt_performance_means = wainer_gt_performance_df.groupby(
    ['question_image', 'question', 'chart_type', 'test_type', 'modality']
)['is_correct'].mean().reset_index()
wainer_gt_performance_means['question_id'] = wainer_gt_performance_means['chart_type']


wainer_gt_performance_means_dfs = []
for chart in ['bar', 'line', 'radial']:
    wainer_gt_performance_chart_means = wainer_gt_performance_means[
        wainer_gt_performance_means['chart_type'].isin(['table', chart])
    ].copy()

    wainer_gt_performance_chart_means['question_id'] = (
        wainer_gt_performance_chart_means['question'] + chart
    )
    
    wainer_gt_performance_means_dfs.append(
        wainer_gt_performance_chart_means
    )
wainer_gt_performance_means = pd.concat(wainer_gt_performance_means_dfs)
wainer_gt_performance_pivot = wainer_gt_performance_means.pivot(
    index='question_id', values='is_correct', columns='modality'
).reset_index()


In [127]:
question_category = [
    ('bg-1', 'modified'),
    ('bg-2', 'modified'),
    ('bg-3', 'base'),
    ('bg-4', 'base'),
    ('bg-5', 'modified'),
    ('bg-6', 'modified'),
    ('bg-7', 'modified'),
    ('bg-8', 'base'),
    ('bg-9', 'modified'),
    ('bg-10', 'base'),
    ('bg-11', 'base'),
    ('bg-12', 'base'),
    ##
    ('sp-1', 'modified'),
    ('sp-2', 'base'),
    ('sp-3', 'base'),
    ('sp-4', 'base'),
    ('sp-5', 'modified'),
    ('sp-6', 'modified'),
    ('sp-7', 'modified'),
    ('sp-8', 'modified'),
    ('sp-9', 'modified'),
    ('sp-10', 'base'),
    ('sp-11', 'base'),
    ('sp-12', 'base'),
    ##
    ('lg1-1', 'base'),
    ('lg1-2', 'modified'),
    ('lg1-3', 'modified'),
    ('lg1-4', 'base'),
    ('lg1-5', 'modified'),
    ('lg1-6', 'modified'),
    ('lg1-7', 'base'),
    ('lg1-8', 'base'),
    ('lg1-9', 'base'),
    ('lg1-10', 'modified'),
    ('lg1-11', 'base'),
    ('lg1-12', 'modified'),
]

for i in range(1, 13):
    if i <= 6:
        question_category.append((f'bg-table-{i}', "base"))
        question_category.append((f'sp-table-{i}', "base"))
        question_category.append((f'lg1-table-{i}', "base"))
    else:
        question_category.append((f'bg-table-{i}', "modified"))
        question_category.append((f'sp-table-{i}', "modified"))
        question_category.append((f'lg1-table-{i}', "modified"))

In [128]:
modality_category_df = pd.DataFrame(question_category, columns=['chart_type', 'item_type'])
brbf_gt_performance_df = modality_performance_df[
    modality_performance_df['test_type'].isin(['brbf'])
].copy()

brbf_gt_performance_df['chart_type'] = brbf_gt_performance_df['chart_type'].apply(
    lambda c : c.replace('.png', '')
)

brbf_gt_performance_means = brbf_gt_performance_df.groupby(
    ['question_image', 'task_category', 'chart_type', 'test_type', 'modality']
)['is_correct'].mean().reset_index()

brbf_gt_performance_means = pd.merge(brbf_gt_performance_means, modality_category_df)
brbf_gt_performance_means['question_id'] = (
    brbf_gt_performance_means['task_category'] + '-' +
    brbf_gt_performance_means['item_type']
)
brbf_gt_performance_means['chart_type'] = brbf_gt_performance_means['chart_type'].apply(
    lambda c : '-'.join(c.split('-')[:-1])
)

brbf_gt_performance_means['question_id'] = (
    brbf_gt_performance_means['question_id'] + '-' 
    + brbf_gt_performance_means['chart_type'].apply(lambda c : c.replace("-table", ""))
)

brbf_gt_performance_means_pivot = brbf_gt_performance_means.pivot(
    index='question_id', columns='modality' , values='is_correct'
).reset_index()

wainer_gt_performance_pivot['test_type'] = 'wainer'
brbf_gt_performance_means_pivot['test_type'] = 'brbf'

table_graph_performance_means = pd.concat([
    wainer_gt_performance_pivot,
    brbf_gt_performance_means_pivot
])
table_graph_performance_means = table_graph_performance_means.melt(
    id_vars=['question_id', 'test_type'], 
    value_vars=['graph', 'table']
)

In [138]:
try:
    modality_ci_df = pd.read_csv("./dataframes/modality_item_level_data.csv")
    modality_item_level_data = pd.read_csv("./dataframes/modality_item_level_data.csv")
    table_graph_comparison_df = pd.read_csv("./dataframes/table_graph_comparison_df.csv")
    print("Reloading modality dataframes")
except:
    print("Regenerating modality dataframes")
    mod_df = modality_performance_df.copy()
    mod_df['task_category'] = mod_df['task_category'].apply(
        lambda t : task_category_map[t] if t in task_category_map.keys() else t
    )
    mod_df['task_category'] = mod_df.apply(
        lambda r : task_category_map[r['question']] if r['question'] in task_category_map.keys() else r['task_category'],
        axis=1
    )
    mod_df['test_modality'] = mod_df['modality']  + ' - ' + mod_df['test_type']

    
    modality_item_level_data, modality_ci_df = create_confidence_interval_df(
        mod_df,orrect',
        id_col='question_image',
        condition_col='test_modality',
    )
    
    modality_ci_df = modality_ci_df.rename(columns={'category': 'modality'})
    modality_ci_df['test_type'] = modality_ci_df['modality'].apply(lambda m : m.split(" - ")[1])
    modality_ci_df['modality'] = modality_ci_df['modality'].apply(lambda m : m.split(" - ")[0])

    table_graph_comparison_df = table_graph_performance_means.pivot(
        index='question_id', columns='modality' , values='value'
    ).reset_index()
    
    modality_ci_df.to_csv("./dataframes/modality_ci_df.csv")
    modality_item_level_data.to_csv("./dataframes/modality_item_level_data.csv")
    table_graph_comparison_df.to_csv("./dataframes/table_graph_comparison_df.csv")

Regenerating modality dataframes


Figure display

In [139]:
modality_ci_df

,modality,mean,ci_upper,ci_lower,test_type
0,graph,0.754912,0.832933,0.663850,wainer
1,table,0.843338,0.928991,0.743666,wainer
2,graph,0.684159,0.751327,0.612969,brbf
3,table,0.690718,0.761556,0.616954,brbf


In [140]:

domain = ['table', 'graph']
color_domain = [ assessment_color_map['wainer'], assessment_color_map['brbf']]
test_domain = ['wainer', 'brbf']


chart = alt.Chart(
    table_graph_performance_means
).mark_line(opacity=0.2).encode(
    x='modality:N',
    y='value',
    detail='question_id',
    color=alt.Color('test_type', scale=alt.Scale(domain=test_domain, range=color_domain),)
)

point_plot = alt.Chart(
    table_graph_performance_means
).mark_circle(opacity=0.1).encode(
    x='modality:N',
    y='value',
    color=alt.Color('test_type',legend=None, scale=alt.Scale(domain=test_domain, range=color_domain)),
    size=alt.value(30)
)

offset_domain = [30, 45]

ci_point_plot =  alt.Chart(modality_ci_df).mark_line(strokeWidth=3, opacity=1).encode(
    x=alt.X('modality:N', title='Category', scale=alt.Scale(domain=domain)),
    y=alt.Y('mean:Q', title='Prop. Correct'),
    opacity=alt.value(1),
    xOffset=alt.XOffset('test_type', scale=alt.Scale(domain=test_domain, range=offset_domain)),
    color=alt.Color(
        'test_type', 
        scale=alt.Scale(domain=test_domain, range=color_domain),
        legend=None
    ),
) + alt.Chart(modality_ci_df).mark_rule(strokeWidth=3, opacity=1).encode(
    x=alt.X('modality:N'),
    y=alt.Y('ci_lower:Q'),
    y2='ci_upper:Q',
    xOffset=alt.XOffset('test_type', scale=alt.Scale(domain=test_domain, range=offset_domain)),
    color=alt.Color(
        'test_type', 
        scale=alt.Scale(domain=test_domain, range=color_domain),
        legend=None
    ),
    detail='modality'
) + alt.Chart(modality_ci_df).mark_point(filled=True, size=75, fill='white', strokeWidth=2).encode(
    x=alt.X('modality:N', title='Category', scale=alt.Scale(domain=domain)),
    y=alt.Y('mean:Q', title='Prop. Correct'),
    opacity=alt.value(1),
    xOffset=alt.XOffset('test_type', scale=alt.Scale(domain=test_domain, range=offset_domain)),
    stroke=alt.Color(
        'test_type', 
        scale=alt.Scale(domain=test_domain, range=color_domain),
        legend=None
    ),
    # detail='modality'
)

modality_plot = (chart + point_plot + ci_point_plot).resolve_scale(color='independent', xOffset='independent')
modality_plot = modality_plot.properties(width=150, height=200, title=f"Table vs Graph")
modality_plot.save("./figures/modality_performance.pdf")
modality_plot

alt.LayerChart(...)

In [141]:
modality_ci_df

,modality,mean,ci_upper,ci_lower,test_type
0,graph,0.754912,0.832933,0.663850,wainer
1,table,0.843338,0.928991,0.743666,wainer
2,graph,0.684159,0.751327,0.612969,brbf
3,table,0.690718,0.761556,0.616954,brbf


In [142]:
def create_pairwise_agent_heatmap(df, x, y, domain, units_of_measure, include_text=True, text_format=".3f"):

    base = alt.Chart(df).mark_rect().encode(
        x=alt.X(x, scale=alt.Scale(domain=domain)),  
        y=alt.Y(y, scale=alt.Scale(domain=domain)),
    )

    color_domain=[-1,1]
    color_reverse = False
    color_condition = alt.condition(
        alt.datum[units_of_measure] < 0.5,
        alt.value('black'),
        alt.value('white')
    )


    height=300
    width=400 
    title=f"Accuracy Vector Correlation between Participants"

    heatmap = base.mark_rect().encode(
        color=alt.Color(f'{units_of_measure}:Q', 
                        legend=None, 
                        scale=alt.Scale(
                            scheme="redgrey",
                            domain=color_domain,
                            reverse=color_reverse)
                       ),
    )

    text = base.mark_text(baseline='middle').encode(
        alt.Text(f'{units_of_measure}:Q', format=text_format),
        color=color_condition
    )

    chart = heatmap.properties(
        width=width,
        height=height,
        title=title
    ) 
    if include_text:
        chart = chart + text

    return chart

### Comparison

In [143]:
from scipy.stats import ttest_rel

In [144]:
table_graph_comparison_df = pd.read_csv("./dataframes/table_graph_comparison_df.csv")
ttest_rel(table_graph_comparison_df['graph'], table_graph_comparison_df['table'])

TtestResult(statistic=-1.4230257645359137, pvalue=0.1599950523321966, df=59)

In [88]:
# table_graph_comparison_df

### Difference of means

In [89]:
def bootstrap_diff_of_mean_ci(
        data: pd.DataFrame,
        measure,
        id_col,
        n_iterations=1000,
        statistic=np.mean
    ):
    """
    
    """
    
    items = list(data[id_col].unique())
    n_size = len(items)
    df = data.copy()

    def bootstrap_iteration(data, chosen_items):
        # filter_df = data[data[id_col].isin(chosen_items)] # Filter based on chosen questions
        filter_df = pd.concat(
            [data[data[id_col].isin([item])] for item in chosen_items]
        )
        
        bs_mean = statistic(filter_df[measure]) 
        return (bs_mean, list(chosen_items))

    
    qset_means1 = Parallel(n_jobs=-1)(
        delayed(bootstrap_iteration)(
            df[df['modality'] == 'table'].copy(),
            rng.choice(items, n_size,  replace=True)
        ) for _ in range(n_iterations)
    )

    qset_means2 = Parallel(n_jobs=-1)(
        delayed(bootstrap_iteration)(
            df[df['modality'] == 'graph'].copy(),
            rng.choice(items, n_size,  replace=True)
        ) for _ in range(n_iterations)
    )
    
    means = []
    qs_used = []
    means1 = [bs_mean for bs_mean, chosen_qs in qset_means1]
    means2 = [bs_mean for bs_mean, chosen_qs in qset_means2]
    means = np.subtract(means1, means2)
 
    # 95% confidence interval
    lower = np.percentile(means, 2.5)
    upper = np.percentile(means, 97.5)
    
    return lower, upper

def create_diff_of_mean_ci_df(
    data: pd.DataFrame,
    measure,
    id_col,
    statistic=np.mean
):
    """ create the dataframe for 95% bootstrapped confidence interval
    data: dataset
    measure: dependent variable
    id_col: units to bootstrap along (e.g. participant_d or item_id)
    condition_col: the different conditions of the experiments
    """
    
    data_list = []


    lower, upper = bootstrap_diff_of_mean_ci(data, measure=measure, statistic=statistic, id_col=id_col)
    mean1 = data[data['modality'] == 'table'][measure].mean()
    mean2 = data[data['modality'] == 'graph'][measure].mean()
    mean = mean1 - mean2
    
    data_list.append({
        "mean": mean,
        "ci_upper": upper, 
        "ci_lower": lower,
    })

    # item_level_data = data.rename(columns={
    #     condition_col: 'category', 
    # }).groupby([id_col, 'category'])[measure].mean().reset_index()

    return pd.DataFrame(data_list)

In [90]:
diff_of_mean_modality_ci_df = create_diff_of_mean_ci_df(
    mod_df[mod_df['test_type'] == 'brbf'],
    measure='is_correct',
    id_col='question_image',
)
diff_of_mean_modality_ci_df

,mean,ci_upper,ci_lower
0,0.00656,0.10897,-0.097392


In [91]:
diff_of_mean_modality_ci_df

,mean,ci_upper,ci_lower
0,0.00656,0.10897,-0.097392


In [92]:
diff_of_mean_modality_ci_df = create_diff_of_mean_ci_df(
    mod_df[mod_df['test_type'] == 'wainer'],
    measure='is_correct',
    id_col='question_image',
)
diff_of_mean_modality_ci_df

,mean,ci_upper,ci_lower
0,0.088427,0.216548,-0.052689


In [93]:
# mod_df['test_type'].unique()
# mod_df[mod_df['test_type'] == 'brbf']['is_correct'].mean()

# Model Fit

# Model comparison fit

## Load R + Libs

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("lme4")
library(lme4)

In [ ]:
%%R
install.packages("MuMIn")
library(MuMIn)

In [ ]:
%%R
install.packages("dplyr")
library(dplyr)

## Create dataframe for models

In [ ]:
def add_brbf_categories(r):
    if (r['test_type'] == 'brbf'):
        return '-'.join(r['image_file'].split("-")[:-1])
    return r['chart_type']


all_df = performance_df.copy()
all_df['chart_type_filled'] = all_df.apply(add_brbf_categories, axis=1)
all_df['chart_type'] = all_df['chart_type_filled'].replace(chart_categories)
all_df = all_df[all_df['chart_type'] != 'Table']

all_df['task_category'] = all_df['task_category'].apply(
    lambda t : task_category_map[t] if t in task_category_map.keys() else t
)
all_df['task_category'] = all_df.apply(
    lambda r : task_category_map[r['question']] if r['question'] in task_category_map.keys() else r['task_category'],
    axis=1
)

all_df['item_id'] = all_df['question_image']
all_df = all_df[['test_type', 'chart_type', 'task_category', 'item_id', 'is_correct']]
initial_df = all_df.copy()

print(
    all_df['chart_type'].unique(), 
    all_df['task_category'].unique(), 
    all_df['test_type'].unique()
)

## Model Comparisons

In [ ]:
%%R

# define options for all models
control <- glmerControl(
    optimizer = "bobyqa",
    optCtrl = list(maxfun = 2e5)
)

### Null model

In [ ]:
%%R -i all_df

null_model <- glmer(
    is_correct ~ 1 + (1 | item_id), 
    data = all_df, 
    family = binomial, 
    control = control
)

### Comparing test fixed effect to null model

In [ ]:
%%R -i all_df

test_model <- glmer(
    is_correct ~ test_type + (1 | item_id), 
    data = all_df, 
    family = binomial,
    control = control
)

anova(null_model, test_model, test="LRT")

In [ ]:
%%R

r.squaredGLMM(test_model, null_model)

### Comparing task type fixed effect to null model

In [ ]:
%%R -i all_df

task_model <- glmer(
    is_correct ~ task_category + (1 | item_id), 
    data = all_df, 
    family = binomial, 
    control = control
)

anova(null_model, task_model, test="LRT")

### Comparing graph type fixed effect to null model

In [ ]:
%%R -i all_df

control <- glmerControl(
    optimizer = "bobyqa",
    optCtrl = list(maxfun = 2e5)
)

graph_model <- glmer(
    is_correct ~ chart_type + (1 | item_id), 
    data = all_df, 
    family = binomial,
    control = control
)

anova(null_model, graph_model, test="LRT")

### Comparing graph/test type fixed effect to item model

In [ ]:
%%R -i all_df

control <- glmerControl(
    optimizer = "bobyqa",
    optCtrl = list(maxfun = 2e5)
)

task_model <- glmer(
    is_correct ~ task_category + (1 | item_id), 
    data = all_df, 
    family = binomial,
    control = control
)

task_graph_model <- glmer(
    is_correct ~ task_category + chart_type + (1 | item_id), 
    data = all_df, 
    family = binomial,
    control = control
)

task_test_model <- glmer(
    is_correct ~ test_type + task_category + (1 | item_id), 
    data = all_df, 
    family = binomial,
    control = control
)

In [ ]:
%%R
anova(task_model, task_graph_model, test="LRT")

In [ ]:
%%R
anova(task_model, task_test_model, test="LRT")

In [ ]:
# all_df

### Find model fit + variance

In [ ]:
%%R -i all_df

control <- glmerControl(
    optimizer = "bobyqa",
    optCtrl = list(maxfun = 2e5)
)

# Logistic regression with item as a fixed effect
null_model <- glmer(is_correct ~ 1 + (1 | item_id), data = all_df, family = binomial, control = control)
test_model <- glmer(is_correct ~ test_type + (1 | item_id), data = all_df, family = binomial, control = control)
task_model <- glmer(is_correct ~ task_category + (1 | item_id), data = all_df, family = binomial, control = control)
graph_model <- glmer(is_correct ~ chart_type + (1 | item_id), data = all_df, family = binomial, control = control)
graph_test_model <- glmer(is_correct ~ test_type + chart_type + (1 | item_id), data = all_df, family = binomial, control = control)
graph_task_model <- glmer(is_correct ~ task_category + chart_type + (1 | item_id), data = all_df, family = binomial, control = control)
task_test_model <- glmer(is_correct ~ test_type + task_category + (1 | item_id), data = all_df, family = binomial, control = control)
graph_task_test_model <- glmer(is_correct ~ test_type + task_category + chart_type + (1 | item_id), data = all_df, family = binomial, control = control)

In [ ]:
%%R -i all_df

control <- glmerControl(
    optimizer = "bobyqa",
    optCtrl = list(maxfun = 2e5)
)

interaction_model <- glmer(
    is_correct ~ test_type*task_category + chart_type + (1 | item_id), 
    data = all_df, family = binomial,
    control = control
)

In [ ]:
%%R -i all_df

results <- data.frame(
  Model = character(),
  Marginal_R2 = numeric(),
  Conditional_R2 = numeric(),
  BIC = numeric(),
  AIC = numeric(),
  bootstap_iter = numeric(),
  stringsAsFactors = FALSE
)

# List of models with their formulas
model_formulas <- list(
null_model = is_correct ~ (1 | item_id),
test_model = is_correct ~ test_type + (1 | item_id),
task_model = is_correct ~ task_category + (1 | item_id),
graph_model = is_correct ~ chart_type + (1 | item_id),
graph_test_model = is_correct ~ test_type + chart_type + (1 | item_id),
graph_task_model = is_correct ~ task_category + chart_type + (1 | item_id),
task_test_model = is_correct ~ test_type + task_category + (1 | item_id),
graph_task_test_model = is_correct ~ test_type + task_category + chart_type + (1 | item_id),
interaction_model = is_correct ~ test_type * task_category + chart_type + (1 | item_id)
)

for (model_name in names(model_formulas)) {
formula <- model_formulas[[model_name]]
warning_occurred <- FALSE

model <- tryCatch({
  fit <- glmer(
    formula,
    data = all_df,
    family = binomial,
    control = control
  )
  fit
}, warning = function(w) {
  if (grepl("fixed-effect model matrix is rank deficient", w$message)) {
    warning_occurred <<- TRUE
  }
  invokeRestart("muffleWarning")
})

# Skip to the next iteration if model fitting failed
if (inherits(model, "error")) next

# Compute R-squared, BIC, and AIC
r_squared <- r.squaredGLMM(model, null_model)
bic_value <- BIC(model)
aic_value <- AIC(model)

# Append results to the data frame
results <- rbind(results, data.frame(
  Model = model_name,
  Marginal_R2 = r_squared[1],
  Conditional_R2 = r_squared[2],
  BIC = bic_value,
  AIC = aic_value,
  bootstap_iter = i,
  Warning = warning_occurred  # Store whether warning occurred
))
}

write.csv(results, "./dataframes/model_fit_fulldata.csv", row.names = FALSE)

In [ ]:
# %%R
# results

In [ ]:
initial_df = all_df.copy()

In [ ]:
len(initial_df)

### Bootstrap model fit

In [ ]:
%%R -i initial_df

control <- glmerControl(
    optimizer = "bobyqa",
    optCtrl = list(maxfun = 2e5)
)

unique_items <- unique(initial_df$item_id)

# Initialize an empty data frame to store the results
results <- data.frame(
  Model = character(),
  Marginal_R2 = numeric(),
  Conditional_R2 = numeric(),
  BIC = numeric(),
  AIC = numeric(),
  bootstap_iter = numeric(),
  stringsAsFactors = FALSE
)

all_sampled_items <- list()

for (i in 1:100) {
  print(i)
  sampled_items <- sample(unique_items, size = length(unique_items), replace = TRUE)
  all_sampled_items <- c(all_sampled_items, list(sampled_items))

  all_df <- do.call(rbind, lapply(sampled_items, function(item) {
    initial_df[initial_df$item_id == item, ]
  }))

  null_model <- glmer(
    is_correct ~ 1 + (1 | item_id), 
    data = all_df, 
    family = binomial, 
    control = control
  )

  # List of models with their formulas
  model_formulas <- list(
    null_model = is_correct ~ (1 | item_id),
    test_model = is_correct ~ test_type + (1 | item_id),
    task_model = is_correct ~ task_category + (1 | item_id),
    graph_model = is_correct ~ chart_type + (1 | item_id),
    graph_test_model = is_correct ~ test_type + chart_type + (1 | item_id),
    graph_task_model = is_correct ~ task_category + chart_type + (1 | item_id),
    task_test_model = is_correct ~ test_type + task_category + (1 | item_id),
    graph_task_test_model = is_correct ~ test_type + task_category + chart_type + (1 | item_id),
    interaction_model = is_correct ~ test_type * task_category + chart_type + (1 | item_id)
  )

  for (model_name in names(model_formulas)) {
    formula <- model_formulas[[model_name]]
    warning_occurred <- FALSE

    model <- tryCatch({
      fit <- glmer(
        formula,
        data = all_df,
        family = binomial,
        control = control
      )
      fit
    }, warning = function(w) {
      if (grepl("fixed-effect model matrix is rank deficient", w$message)) {
        warning_occurred <<- TRUE
      }
      invokeRestart("muffleWarning")
    })

    # Skip to the next iteration if model fitting failed
    if (inherits(model, "error")) next

    # Compute R-squared, BIC, and AIC
    r_squared <- r.squaredGLMM(model, null_model)
    bic_value <- BIC(model)
    aic_value <- AIC(model)

    # Append results to the data frame
    results <- rbind(results, data.frame(
      Model = model_name,
      Marginal_R2 = r_squared[1],
      Conditional_R2 = r_squared[2],
      BIC = bic_value,
      AIC = aic_value,
      bootstap_iter = i,
      Warning = warning_occurred  # Store whether warning occurred
    ))
  }
}

write.csv(results, "./figures/model_statistics2.csv", row.names = FALSE)

In [ ]:
%%R

my_list <- all_sampled_items
num_columns <- max(sapply(my_list, length))
column_names <- paste0("Column_", seq_len(num_columns))

df <- do.call(rbind, lapply(my_list, function(x) {
  length(x) <- num_columns
  x
}))

# Assign column names to the dataframe
colnames(df) <- column_names

# Convert to a data.frame and ensure appropriate types
df <- as.data.frame(df, stringsAsFactors = FALSE)

# Save the dataframe as a CSV file
write.csv(df, "./model_bootstrapped_items.csv", row.names = FALSE)

In [ ]:
model_bootstrapped_items = pd.read_csv("model_bootstrapped_items.csv")

### Plot Model Fit

In [ ]:
valid_interaction_combos = all_df[['item_id', 'test_type', 'task_category']].drop_duplicates()
valid_interaction_combos

In [ ]:
# items

In [ ]:
invalid_interaction_runs = []
for i, row in pd.read_csv("./model_bootstrapped_items.csv").iterrows():
    item = pd.DataFrame({'item_id': row.to_list()}).drop_duplicates()
    test_task_interaction_counts = pd.merge(valid_interaction_combos, item)[['test_type', 'task_category']].value_counts()
    test_task_interaction_counts = test_task_interaction_counts.reset_index()
    test_task_interaction_counts = test_task_interaction_counts.pivot(index='test_type', columns='task_category', values='count')

    if (len(test_task_interaction_counts) != len(test_task_interaction_counts.dropna())):
        invalid_interaction_runs.append(i)
        print("Invalid bootstrap for interaction model found: Row ", i)
print("Total runs invalid: ", len(invalid_interaction_runs))

In [ ]:
model_statistics_allbs = pd.read_csv("./figures/model_statistics2.csv")
model_statistics_allbs = model_statistics_allbs[~model_statistics_allbs['bootstap_iter'].isin(invalid_interaction_runs)]
len(model_statistics_allbs['bootstap_iter'].unique())

In [ ]:
model_statistics_allbs

In [ ]:
null_model = model_statistics_allbs[model_statistics_allbs['Model'] == 'null_model']

def get_bootstrap_iter_aic(r):
    aic = r['AIC']
    null_aic = null_model[null_model['bootstap_iter'] == r['bootstap_iter']]['AIC'].iloc[0]
    return aic - null_aic

model_statistics_allbs['relative_aic'] = model_statistics_allbs.apply(
    get_bootstrap_iter_aic,
    axis=1
)

In [ ]:
model_statistics = model_statistics_allbs.copy()
model_statistics = model_statistics[model_statistics['Model'] != 'null_model']
model_statistics_ci = model_statistics.groupby('Model').agg(
    bic_mean=('BIC', 'mean'),
    bic_std=('BIC', 'std'),
    relative_aic_mean=('relative_aic', 'mean'),
    relative_aic_std=('relative_aic', 'std'),
    mr2_mean=('Marginal_R2', 'mean'),
    mr2_std=('Marginal_R2', 'std'),
    mr2_ci_upper=('Marginal_R2', lambda m : np.percentile(m, 97.5)),
    mr2_ci_lower=('Marginal_R2', lambda m : np.percentile(m, 2.5 )),
    cr2_mean=('Conditional_R2', 'mean'),
    cr2_std=('Conditional_R2', 'std'),
).reset_index()

model_statistics_ci['mr2_ystd'] = model_statistics_ci['mr2_mean'] - model_statistics_ci['mr2_std']
model_statistics_ci['mr2_ystd2'] = model_statistics_ci['mr2_mean'] + model_statistics_ci['mr2_std']

model_statistics_ci['cr2_ystd'] = model_statistics_ci['cr2_mean'] - model_statistics_ci['cr2_std']
model_statistics_ci['cr2_ystd2'] = model_statistics_ci['cr2_mean'] + model_statistics_ci['cr2_std']

model_statistics_ci['relative_aic_ystd'] = model_statistics_ci['relative_aic_mean'] - model_statistics_ci['relative_aic_std']
model_statistics_ci['relative_aic_ystd2'] = model_statistics_ci['relative_aic_mean'] + model_statistics_ci['relative_aic_std']

# replace means with full data fit
model_fit_on_fulldata = pd.read_csv("./dataframes/model_fit_fulldata.csv")
model_fit_on_fulldata = model_fit_on_fulldata.rename(columns={'Marginal_R2': 'mr2_fulldata'})
model_statistics_ci = pd.merge(model_statistics_ci, model_fit_on_fulldata[['Model', 'mr2_fulldata']])

In [ ]:
# model_fit_on_fulldata
model_statistics_ci

In [ ]:
# model_fit_on_fulldata

In [ ]:
# model_statistics_ci

#### Get Split-half R^2 

In [ ]:
split_half_correlations = []
for _ in range(10000):
    correlation_df = all_df.sample(frac=1, replace=False)
    half_one = correlation_df.iloc[0:(len(all_df) // 2)].groupby('item_id')['is_correct']
    half_one = half_one.mean().reset_index().sort_values(by='item_id')
    
    half_two = correlation_df.iloc[(len(all_df) // 2):len(all_df)].groupby('item_id')['is_correct']
    half_two = half_two.mean().reset_index().sort_values(by='item_id')
    
    iter_corr = np.corrcoef(half_one['is_correct'], half_two['is_correct'])
    split_half_correlations.append(iter_corr[0][1]**2)

split_half_mean = np.mean(split_half_correlations)
split_half_ci_lower, split_half_ci_upper = np.percentile(split_half_correlations, [2.5, 97.5])
split_half_df = pd.DataFrame({
    'index': [0],
    'mean': split_half_mean,
    'ci_upper': split_half_ci_upper,
    'ci_lower': split_half_ci_lower,
})

In [ ]:
split_half_df

In [ ]:
model_order = [
    'test_model',
    'task_model',
    'graph_model',
    'task_test_model',
    'graph_test_model',
    'graph_task_model',     
    'graph_task_test_model',
    'interaction_model'
]

line_path = "M -3.5,0 L 3.5,0"

def plot_model_fit(measure, yscale):
    mean_model_plot = alt.Chart(model_statistics_ci).mark_point(color='black', shape=line_path).encode(
        x=alt.X('Model:N', scale=alt.Scale(domain=model_order)),
        y=alt.Y(f'{measure}_fulldata:Q', scale=alt.Scale(domain=yscale), title=""),
        strokeWidth=alt.value(2)
    )
    
    std_model_plot = alt.Chart(model_statistics_ci).mark_bar(color='black', opacity=0.1).encode(
        x=alt.X('Model:N', scale=alt.Scale(domain=model_order)),
        y=alt.Y(f'{measure}_ci_upper:Q'),
        y2=(f'{measure}_ci_lower:Q'),
        size=alt.value(20)
    )

    split_half_band = alt.Chart(split_half_df).mark_rect(color='black', opacity=0.2).encode(
        y=alt.Y(f'ci_upper:Q'),
        y2=(f'ci_lower:Q'),
    )

    split_half_mean = alt.Chart(split_half_df).mark_rule(color='black', strokeDash=[5, 5]).encode(
        y=alt.Y(f'mean:Q'),
        strokeWidth=alt.value(1)
    )
    
    model_fit_figure = (mean_model_plot + std_model_plot + split_half_band + split_half_mean).properties(
        height=200,
        width=200
    )

    return model_fit_figure

# mr2_fulldata
model_fit_figure = plot_model_fit('mr2', yscale=[0, 1])
model_fit_figure.save("./figures/model_fit.pdf")
model_fit_figure

In [ ]:
model_statistics_ci[['Model', 'mr2_fulldata', 'mr2_ci_upper', 'mr2_ci_lower']]

In [ ]:
temp = item_df[['test_type', 'task_category']].value_counts().reset_index()
temp.pivot(index='test_type', columns='task_category', values='count')

In [ ]:
model_statistics = pd.read_csv("./figures/model_statistics.csv")